# Junction Parameters 

$$E_J = \frac{\hbar}{2e}{I_c}$$
where critical current goes linearly with area and inversely prp to oxide thickness

$$I_C = \frac{\pi \Delta(0)}{2eR_n}$$
where \(\Delta(0) = 170 \mu\text{eV}\) for Aluminum 

$$\omega = \sqrt{8E_CE_J} - E_C$$ 
for transmon


In [130]:
critical_current_per_area = 0.2 # micro A /um^2
junction_length = 0.35 # currently we are getting 0.25 um 


In [131]:
# m,akeimport numpy as np

In [132]:
import numpy as np
# given the calculations below, compute the bridge width given angle, undercut length and junction length
def compute_bridge_width(junction_length, undercut_length, angle_deg):
    """
    Compute the bridge width given junction length, undercut length, and angle.
    Returns the bridge width in um.
    """
    dep_width = undercut_length * np.tan(np.radians(angle_deg))
    bridge_width = 2 * dep_width - junction_length
    return bridge_width

bridge_width = compute_bridge_width(junction_length = junction_length, undercut_length= 0.800, angle_deg = 15.7)
print(f"Bridge width: {bridge_width:.2f} um")
gap = junction_length


Bridge width: 0.10 um


# What is the \(E_J\) I need?

In [133]:
# given E_c  and omeaga find E_J
Ec = 0.143 # GHz
target_omega = 3.5 # GHz
is_squid = False
def find_E_J(E_c, omega):
    """
    Find E_J given E_c and omega.
    """
    return ((omega-E_c)**2) / (8 * E_c)

targ_Ej = find_E_J(Ec, target_omega)
if is_squid:
    # for squid we need to divide by 2
    targ_Ej /= 2
print(f"targ_Ej = {targ_Ej:.2f} GHz")

targ_Ej = 9.85 GHz


In [144]:

def find_L_J(E_J_freq):
    """
    Calculate Josephson inductance L_J from Josephson energy frequency E_J_freq (in GHz).

    L_J = Φ₀² / (4 * π² * h * E_J_freq)
    where:
        Φ₀ = flux quantum = 2.068e-15 Wb
        h = Planck's constant = 2π * ħ
    """
    phi0 = 2.068e-15  # Wb
    h = 6.62607015e-34  # J*s
    E_J_Hz = E_J_freq * 1e9  # Convert GHz to Hz
    L_J = phi0**2 / (4 * np.pi**2 * h * E_J_Hz)
    return L_J

L_J = find_L_J(9.85)
print(f"L_J = {L_J * 1e9:.2e} nH")
    

L_J = 1.66e+01 nH


In [140]:
# convert C into E_C
def capacitance_from_Ec(Ec):
    """
    Convert charging energy E_c (in GHz) to capacitance C (in Farads).
    E_c = e^2 / (2C * hbar)
    """
    e = 1.602176634e-19  # C
    hbar = 1.0545718e-34  # J*s
    Ec_J = Ec * 1e9 * hbar  # Convert GHz to Joules
    C = e**2 / (2 * Ec_J)
    return C

C = capacitance_from_Ec(Ec = 0.143)
print(f"Capacitance C = {C*1e15:.2f} fF")

Capacitance C = 851.10 fF


In [145]:
1/np.sqrt(L_J*C) * 1e-9 # convert to GHz

8.413683357203336

# What is the pin size I need ? 
given a gap of 0.25 microns

In [134]:
import numpy as np

# Use this $$E_J = \frac{\hbar}{2e}{I_c}$$  and critical current per area to find the pin size using a function
def pin_size_from_Ej(E_j, critical_current_per_area, gap):
    """
    Calculate the pin size (area) needed for a given E_j.
    E_j: Josephson energy in GHz
    critical_current_per_area: in microA/um^2
    gap: in um (not used in calculation, just for reference)
    Returns: area in um^2
    """
    # Constants
    hbar = 1.0545718e-34  # J*s
    e = 1.602176634e-19   # C
    GHz_to_J = 6.62607015e-34  # Planck's constant (J*s) * 1 Hz

    # Convert E_j from GHz to Joules
    E_j_J = E_j * GHz_to_J * 1e9

    # Calculate I_c from E_j
    I_c = (2 * e * E_j_J) / hbar  # in Amps

    # Convert critical_current_per_area to A/um^2
    critical_current_per_area_A = critical_current_per_area * 1e-6

    # Area in um^2
    area = I_c / critical_current_per_area_A
    print(area)
    return area/gap

pin_length = pin_size_from_Ej(targ_Ej, critical_current_per_area, gap)
print(f"Required pin length: {pin_length:.2f} um")

0.09916694342714646
Required pin length: 0.28 um


# What RT resistance will I measure?

In [135]:
# Use these formilas to find resistance of junction $$E_J = \frac{\hbar}{2e}{I_c}$$
# where critical current goes linearly with area and inversely prp to oxide thickness

# $$I_C = \frac{\pi \Delta(0)}{2eR_n}$$
# where \(\Delta(0) = 170 \mu\text{eV}\) for Aluminum 
def calculate_R_n(pin_length, gap, critical_current_per_area):
    """
    Calculate the normal resistance R_n of the junction.

    Formulas used:
    - Area = pin_length × gap
    - Critical current: I_c = critical_current_per_area × Area
    - Josephson relation: E_J = (ħ / 2e) * I_c
    - I_C = (π Δ(0)) / (2e R_n), where Δ(0) = 170 μeV for Aluminum

    Args:
        pin_length: length of the pin in um
        gap: gap in um
        critical_current_per_area: in microA/um^2

    Returns:
        R_n in Ohms
    """
    # Constants
    e = 1.602176634e-19  # C
    Delta_0 = 170e-6 * e  # 170 μeV in Joules

    # Calculate area
    area = pin_length * gap  # um^2

    # Calculate critical current (I_c)
    critical_current_per_area_A = critical_current_per_area * 1e-6  # A/um^2
    I_c = critical_current_per_area_A * area  # A

    # Calculate R_n using I_C = (π Δ(0)) / (2e R_n)
    R_n = (np.pi * Delta_0) / (2 * e * I_c)  # Ohms

    return R_n


R_n = calculate_R_n(pin_length, gap, critical_current_per_area)
print(f"Expected RT resistance R_n: {R_n:.2f} Ω")

Expected RT resistance R_n: 13463.93 Ω


# Summarize the results of whole notebook in a few lines with a list of ciritcial parameters

In [136]:
print("Summary of Critical Parameters:")
print(f"- Target Frequency (ω): {target_omega} GHz")
print(f"- Charging Energy (E_c): {Ec:.3f} GHz")
print(f"- Target Josephson Energy (E_J): {targ_Ej:.2f} GHz")
print(f"- Critical Current per Area: {critical_current_per_area} μA/μm²")
print(f"- Gap: {gap} μm")
print(f"- Required Pin Length: {pin_length:.2f} μm")
print(f"- Expected RT Resistance (R_n): {R_n:.2f} Ω")
print(f"- Bridge Width: {bridge_width:.2f} um")

Summary of Critical Parameters:
- Target Frequency (ω): 3.5 GHz
- Charging Energy (E_c): 0.143 GHz
- Target Josephson Energy (E_J): 9.85 GHz
- Critical Current per Area: 0.2 μA/μm²
- Gap: 0.35 μm
- Required Pin Length: 0.28 μm
- Expected RT Resistance (R_n): 13463.93 Ω
- Bridge Width: 0.10 um
